In [1]:
import pickle
import pandas as pd

#### Identificator to sequence mapping

In [2]:
id2seq = {}
aa1 = "ACDEFGHIKLMNPQRSTVWY"
from Bio import SeqIO
for record in SeqIO.parse("/home/db/pdb/pdb_seqres_clean.txt", "fasta"):
    nonstd_present = False
    for aa in str(record.seq):
        if aa not in aa1:
            nonstd_present = True
            break
    if not nonstd_present:
        id2seq[record.id] = str(record.seq)

#### Get cchmmprof assignments

In [3]:
df = pickle.load(open('./../../1_Data_Preparation/out/pickle/data_all_74.p', 'rb'))

In [4]:
entries = set(df.index.tolist())

In [5]:
cchmmpred_data = {}
for entry in entries:
    f = open('results/%s.profile' % entry, 'r')
    data = f.readlines()
    try:
        prob = data[0].rstrip().split(' ')[7]
        cc_prob = 0
        if float(prob) >= 0.5:
            cc_prob = 1
        assignment = data[1][5:].rstrip()
        assignment = assignment.replace('H', '1')
        assignment = assignment.replace('.', '0')
        assert(len(assignment) == len(id2seq[entry]))
        cchmmpred_data[entry] = [assignment, cc_prob]
    except IndexError:
        print("Error: %s" % (entry))

#### Check everything's fine

In [6]:
len(entries), len(cchmmpred_data)

(28938, 28938)

#### Create pandas

In [7]:
df = pd.DataFrame.from_dict(cchmmpred_data, orient='index')
df.columns = ['cchmmprof_assignment', 'cchmmprof_cc']

#### Save data

In [8]:
df.to_csv('./../../1_Data_Preparation/out/csv/cchmmprof_all_74.csv')
df.to_pickle(open('./../../1_Data_Preparation/out/pickle/cchmmprof_all_74.p', 'wb'))